# Initiate

In [2]:
import time
import random
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException 
from selenium import webdriver
import tqdm

In [2]:
s=Service('C:/SAINS DATA/msedgedriver.exe')
options = webdriver.EdgeOptions()
options.add_experimental_option("detach", True)
options.add_argument('disable_infobars')
# options.headless = True

In [3]:
news_sources = pd.read_excel(r"C:\CAREER\CESGS\Scraping News\Updated_Daftar_Saham(After).xlsx")
news_sources.head()

,No,Kode,Nama Perusahaan,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,1,AALI,Astra Agro Lestari,8,7,2,5,9,9,9,14,7
1,2,ABBA,Mahaka Media,2,0,1,3,3,4,1,1,2
2,3,ABDA,Asuransi Bina Dana Arta,0,1,0,0,0,0,4,1,1
3,4,ABMM,ABM Investama,0,1,2,0,2,2,9,0,6
4,5,ACES,Ace Hardware Indonesia,7,5,3,5,9,11,15,24,29


# Scraping

In [73]:
driver = webdriver.Edge(service=s, options=options)
url="https://www.google.com/search?q=Astra+Agro+Lestari%0D%0A&sca_esv=563020551&rlz=1C1CHZN_enID921ID921&biw=988&bih=662&tbm=nws&ei=DCWeZvzQDcaZseMPx7er-AM&ved=0ahUKEwj8hrfGqbqHAxXGTGwGHcfbCj84ogIQ4dUDCA0&uact=5&oq=Astra+Agro+Lestari%0D%0A&gs_lp=Egxnd3Mtd2l6LW5ld3MiE0FzdHJhIEFncm8gTGVzdGFyaQpIAFAAWABwAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEC-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz-news"
driver.get(url)

time.sleep(3)

title_list = []
link_list = []
date_list = []
comp = []

the_three_that_left_behind = ['Industri dan Perdagangan Bintraco Dharma',
                                'Malacca Trust Wuwungan Insuran',
                                'Pelayaran Nasional Ekalya Purnamasari']

### SCRAPING ###
for company in tqdm.tqdm(the_three_that_left_behind):
    pagination = 1

    search_area = driver.find_element(By.CSS_SELECTOR, 'textarea[class="gLFyf"]')
    search_area.clear()
    search_area.send_keys(company)
    time.sleep(0.1)
    
    search_button = driver.find_element(By.CSS_SELECTOR, 'button[class="HZVG1b Tg7LZd"]')
    search_button.click()
    
    print(f'Scraping {company}...')
    time.sleep(0.7)
    while True:
        try:
            # Get Links
            links = driver.find_elements(By.CSS_SELECTOR, 'a[class = "WlydOe"]')
            dates = driver.find_elements(By.CSS_SELECTOR, 'div[class = "OSrXXb rbYSKb LfVVr"]')
            titles = driver.find_elements(By.CSS_SELECTOR, 'div[class="n0jPhd ynAwRc MBeuO nDgy9d"]')

            # Ensure lists are of the same length
            length = min(len(links), len(dates), len(titles))

            for i in range(length):
                link_list.append(links[i].get_attribute('href'))
                date_list.append(dates[i].text)
                title_list.append(titles[i].text)
                comp.append(company)
            
            # To the Next Page
            next_page = driver.find_element(By.ID, 'pnnext')
            next_page.click()
            time.sleep(random.randint(1,2))
            
            pagination = pagination + 1
        except NoSuchElementException:
            print(f'Gathered {pagination} page for {company}')
            break
    
    df_news = pd.DataFrame({'Nama Perusahaan':comp, 
                            'Link': link_list,
                            'Title':title_list,
                            'Date': date_list})
    
    
    df_news.to_csv('Scraping Berita Perusahaan.csv', index=False)
    print(f'Scraped total {len(link_list)} news')
    print('-------------------------------------------')
    
print('Done!')
        


  0%|          | 0/3 [00:00<?, ?it/s]

Scraping Industri dan Perdagangan Bintraco Dharma...


 33%|███▎      | 1/3 [00:39<01:18, 39.06s/it]

Gathered 14 page for Industri dan Perdagangan Bintraco Dharma
Scraped total 131 news
-------------------------------------------
Scraping Malacca Trust Wuwungan Insuran...


 67%|██████▋   | 2/3 [00:50<00:22, 22.71s/it]

Gathered 5 page for Malacca Trust Wuwungan Insuran
Scraped total 177 news
-------------------------------------------
Scraping Pelayaran Nasional Ekalya Purnamasari...


100%|██████████| 3/3 [01:29<00:00, 29.74s/it]

Gathered 16 page for Pelayaran Nasional Ekalya Purnamasari
Scraped total 317 news
-------------------------------------------
Done!


In [74]:
df_scraping_temp = pd.read_csv('Scraping Berita Perusahaan.csv')
df_scraping_temp.head()

,Nama Perusahaan,Link,Title,Date
0,Industri dan Perdagangan Bintraco Dharma,https://industri.kontan.co.id/news/industri-da...,Industri dan Perdagangan Bintraco Dharma (CARS...,31 Mar 2024
1,Industri dan Perdagangan Bintraco Dharma,https://market.bisnis.com/read/20240320/192/17...,"Pendapatan Bintraco Dharma (CARS) Tembus Rp6,4...",20 Mar 2024
2,Industri dan Perdagangan Bintraco Dharma,https://emitennews.com/news/perkuat-bisnis-oto...,"Perkuat Bisnis Otomotif, Bintraco Dharma (CARS...",8 Mar 2024
3,Industri dan Perdagangan Bintraco Dharma,https://industri.kontan.co.id/news/penjualan-b...,"Penjualan Bintraco Dharma (CARS) Capai Rp 1,65...",30 Apr 2024
4,Industri dan Perdagangan Bintraco Dharma,https://pasardana.id/news/2024/3/21/akuntan-pu...,Akuntan Publik Soroti 11 Anak Usaha CARS Milik...,21 Mar 2024


In [75]:
df_result = pd.read_excel('Scrape Company News_3.xlsx')
df_result.tail()

,Nama Perusahaan,Link,Title,Date
36679,Zyrexindo Mandiri Buana,https://inet.detik.com/consumer/d-1173020/si-m...,Si Mungil yang Tak Hanya untuk Pemula,28 Jul 2009
36680,Zyrexindo Mandiri Buana,https://www.trenasia.com/laptop-untuk-sekolah-...,Kontroversi Laptop untuk Sekolah (Serial 1): S...,3 Agu 2021
36681,Zyrexindo Mandiri Buana,https://review1st.com/news/bocoran-harga-lapto...,Bocoran Harga Laptop Merah Putih: Dibanderol R...,24 Jul 2021
36682,Zyrexindo Mandiri Buana,https://pekanbaru.tribunnews.com/2012/03/18/zy...,Zyrex Luncurkan LW5825 dan LW5823,18 Mar 2012
36683,Zyrexindo Mandiri Buana,https://www.trenasia.com/laptop-for-schools-co...,Laptop for Schools Controversy in Indonesia (S...,3 Agu 2021


In [76]:
df_result_fix = pd.concat([df_result, df_scraping_temp])
df_result_fix.to_excel('Scrape Company News_3.xlsx', index=False)
print('updated!')

updated!


In [4]:
d1 = pd.read_excel("Scrape Company News_ALL.xlsx", sheet_name='Sheet1')
d2 = pd.read_excel("Scrape Company News_2.xlsx")
d3 = pd.read_excel("Scrape Company News_3.xlsx")

print(len(d1['Nama Perusahaan'].unique()) + len(d2['Nama Perusahaan'].unique()) + len(d3['Nama Perusahaan'].unique()))

902


In [6]:
all_company = pd.concat([d1, d2, d3])
all_company.shape

(144545, 4)

In [24]:
for i in tqdm.tqdm(d3.Date):
    if 'lalu' in i:
        d3.Date.loc[d3.Date == i] = '2024'
    else :
        d3.Date.loc[d3.Date == i] = i.split(' ')[-1]

100%|██████████| 37001/37001 [02:03<00:00, 300.49it/s]


In [25]:
d3.to_excel('Scrape Company News_Date Simplified_3.xlsx', index=False)
print('Saved!')

Saved!


In [80]:
all_company.to_csv('Scrape Company News_ALL.csv', index=False)
print('Saved!')

Saved!


In [72]:
# Check for missing data
perusahaan = list(d1['Nama Perusahaan'].unique()) + list(d2['Nama Perusahaan'].unique()) + list(d3['Nama Perusahaan'].unique())
set(news_sources['Nama Perusahaan']) - set(perusahaan)

{'Industri dan Perdagangan Bintr',
 'Malacca Trust Wuwungan Insuran',
 'Pelayaran Nasional Ekalya Purn'}

# Processing

In [27]:
df_company1 = pd.read_excel('Scrape Company News_Date Simplified_ALL.xlsx', sheet_name='Sheet1')
df_company2 = pd.read_excel('Scrape Company News_Date Simplified_ALL.xlsx', sheet_name='Sheet2')
df_company3 = pd.read_excel('Scrape Company News_Date Simplified_ALL.xlsx', sheet_name='Sheet3')

df_company_all = pd.concat([df_company1, df_company2, df_company3])
df_company_all.head()

,Nama Perusahaan,Link,Title,Date
0,Astra Agro Lestari,https://catatanriau.com/news/detail/20346/sida...,"Sidang Pemeriksaan Setempat, Perkara Masyaraka...",2024
1,Astra Agro Lestari,https://ajaib.co.id/harga-saham-aali-layak-ata...,"Bedah Saham AALI, Layak atau Tidak untuk Dikol...",2024
2,Astra Agro Lestari,https://insight.kontan.co.id/news/astra-agro-l...,Astra Agro Lestari (AALI) Fokus Pertahankan Pr...,2024
3,Astra Agro Lestari,https://www.infosawit.com/2024/07/11/pt-astra-...,"PT Astra Agro Lestari Tbk., Resmi Bergabung ke...",2024
4,Astra Agro Lestari,https://finance.detik.com/industri/d-7434671/a...,Astra Agro Lestari Resmi Daftar Jadi Anggota RSPO,2024


In [33]:
df_company_all_filter = df_company_all.drop(df_company_all[df_company_all.Date < 2016].index)
df_company_all_filter.Date.value_counts()

Date
2023    39285
2024    35988
2022    21818
2021    13194
2020     7271
2019     6369
2018     4435
2017     2443
2016     1287
Name: count, dtype: int64

In [57]:
df_company_all_filter_pivot = df_company_all_filter[['Nama Perusahaan', 'Date']].pivot_table(columns='Date',
                                                                                            index=['Nama Perusahaan'], 
                                                                                            aggfunc= 'size', 
                                                                                            fill_value=0).reset_index()
df_company_all_filter_pivot

Date,Nama Perusahaan,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,ABM Investama,0,0,0,0,0,3,24,101,172
1,AKR Corporindo,0,0,0,2,0,19,28,115,103
2,Ace Hardware Indonesia,0,0,3,8,15,29,44,94,98
3,Ace Oldfields,0,0,0,0,0,18,7,11,7
4,Acset Indonusa,9,10,14,13,33,31,29,74,20
...,...,...,...,...,...,...,...,...,...,...
897,XL Axiata,0,0,0,0,0,0,0,1,299
898,Yanaprima Hastapersada,0,2,3,2,6,8,6,4,0
899,Yelooo Integra Datanet,0,0,11,7,1,29,43,12,5
900,Yulie Sekuritas Indonesia,0,2,10,5,2,0,5,3,2


In [61]:
df_company_all_filter_pivot.to_excel('Scrape Company News_Pivoted_ALL.xlsx', index=False)
print('Saved')

Saved


# Locate Elements

In [30]:
# Link 
link = driver.find_elements(By.CSS_SELECTOR, 'a[class = "WlydOe"]')
print(len([elem.get_attribute('href') for elem in link]))

10


In [19]:
# Title 
t = driver.find_elements(By.CSS_SELECTOR, 'div[class = "d6cvqb BBwThe"]')
for i in t:
    print(i.text)

In [17]:
# Next Page
t_click = driver.find_element(By.ID, 'pnnext')
t_click.click()

In [35]:
# Date
date = driver.find_elements(By.CSS_SELECTOR, 'div[class = "OSrXXb rbYSKb LfVVr"]')
print([elem.text for elem in date])

['21 Mei 2024', '3 Apr 2024', '3 Mar 2024', '2 Mei 2024', '21 Mei 2024', '6 Mei 2024', '18 Apr 2024', '6 Apr 2024', '12 Apr 2024', '28 Apr 2024']


In [14]:
p = driver.find_elements(By.TAG_NAME, 'p')
for i in p:
    print(i.text)

Jelang FIFA Matchday Indonesia vs Turkmenistan: Indosat Gelar Meet and Greet dengan Egy Maulana CS
TRIBUN-BALI.COM, SURABAYA - Para pemain Timnas Indonesia menggelar meet and greet bersama komunitas, media, dan karyawan Indosat di Ruang Carribean, Hotel Vasa Surabaya, pada Selasa 5 September 2023. 
Meet and greet diselenggarakan Indosat untuk menjawab antusiasme penggemar sepak bola tanah air sebelum Timnas Indonesia berlaga dalam FIFA Match Day menghadapi Turkmenistan di Gelora Bung Tomo, Surabaya pada Jumat 8 September 2023 mendatang.
Sejumlah pemain Timnas yang hadir seperti Dendy Sulistyawan, Egy Maulana Vikri, Fachruddin Aryanto, dan Reza Arya Pratama.
Mereka berbagi cerita mengenai persiapan jelang laga Jumat nanti kepada belasan peserta serta diiringi dengan sesi tanya jawab, dan games.
 Baca juga: PREDIKSI Line Up Timnas Indonesia di Kualifikasi Piala Asia U23, Rafael Struick, Ivar Jenner Starter
Pemain Dewa United Egy Maulana Vikri yang kini dipanggil memperkuat Timnas ini pun